# Lab 06 Softmax Classification
* Softmax를 사용하여분류를 진행합니다.

### 기본 Library 선언 및 Tensorflow 버전 확인

In [84]:
import tensorflow as tf
import numpy as np
tf.enable_eager_execution()
tf.set_random_seed(777)  # for reproducibility
tfe = tf.contrib.eager

In [85]:
x_data = [[1, 2, 1, 1],
          [2, 1, 3, 2],
          [3, 1, 3, 4],
          [4, 1, 5, 5],
          [1, 7, 5, 5],
          [1, 2, 5, 6],
          [1, 6, 6, 6],
          [1, 7, 7, 7]]
y_data = [[0, 0, 1],
          [0, 0, 1],
          [0, 0, 1],
          [0, 1, 0],
          [0, 1, 0],
          [0, 1, 0],
          [1, 0, 0],
          [1, 0, 0]]

#convert into numpy and float format
x_data = np.asarray(x_data, dtype=np.float32)
y_data = np.asarray(y_data, dtype=np.float32)

In [86]:
#dataset을 선언합니다.
# dataset = tf.data.Dataset.from_tensor_slices((x_data, y_data))
# dataset = dataset.repeat().batch(2)

nb_classes = 3 #class의 개수입니다.

In [87]:
#Weight and bias setting

W = tfe.Variable(tf.random_normal([4, nb_classes]), name='weight')
b = tfe.Variable(tf.random_normal([nb_classes]), name='bias')
variables = [W, b]

print(W,b)

<tf.Variable 'weight:0' shape=(4, 3) dtype=float32, numpy=
array([[ 0.7706481 ,  0.37335402, -0.05576323],
       [ 0.00358377, -0.5898363 ,  1.5702795 ],
       [ 0.2460895 , -0.09918973,  1.4418385 ],
       [ 0.3200988 ,  0.526784  , -0.7703731 ]], dtype=float32)> <tf.Variable 'bias:0' shape=(3,) dtype=float32, numpy=array([-1.3080608 , -0.13253094,  0.5513761 ], dtype=float32)>


In [88]:
# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
def softmax_regression(X):
    return tf.nn.softmax(tf.matmul(X, W) + b)

print(softmax_regression(x_data))

tf.Tensor(
[[1.36571955e-02 7.90162385e-03 9.78441238e-01]
 [3.92597765e-02 1.70347411e-02 9.43705440e-01]
 [3.80385160e-01 1.67723209e-01 4.51891690e-01]
 [3.23390663e-01 5.90759777e-02 6.17533386e-01]
 [3.62997412e-06 6.20727292e-08 9.99996305e-01]
 [2.62520462e-02 1.07279727e-02 9.63019967e-01]
 [1.56525111e-05 4.21802781e-07 9.99983907e-01]
 [2.94076904e-06 3.81133276e-08 9.99997020e-01]], shape=(8, 3), dtype=float32)


In [89]:
def loss_fn(X, Y):
    logits = softmax_regression(X)
    loss = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(logits), axis=1))
    return loss

print(loss_fn(x_data, y_data))

tf.Tensor(6.07932, shape=(), dtype=float32)


In [90]:
x = tf.constant(3.0)
with tf.GradientTape() as g:
    g.watch(x)
    y = x * x # x^2
dy_dx = g.gradient(y, x) # Will compute to 6.0
print(dy_dx)

tf.Tensor(6.0, shape=(), dtype=float32)


In [91]:
def grad_fn(X, Y):
    with tf.GradientTape() as tape:
        loss = loss_fn(X, Y)
        grads = tape.gradient(loss, variables)

        return grads

grad_fn(x_data, y_data)

[<tf.Tensor: id=547079, shape=(4, 3), dtype=float32, numpy=
 array([[ 0.06914616, -0.6509784 ,  0.58183223],
        [-1.5221257 , -1.214863  ,  2.7369888 ],
        [-1.2473826 , -1.7611003 ,  3.008483  ],
        [-1.2014605 , -1.8659232 ,  3.0673838 ]], dtype=float32)>,
 <tf.Tensor: id=547077, shape=(3,), dtype=float32, numpy=array([-0.15212911, -0.342192  ,  0.49432108], dtype=float32)>]

In [95]:
def fit(X, Y, epochs=2000, verbose=100):
    optimizer =  tf.train.GradientDescentOptimizer(learning_rate=0.1)

    for i in range(epochs):
        grads = grad_fn(X, Y)
        optimizer.apply_gradients(zip(grads, variables))
        if (i==0) | ((i+1)%verbose==0):
            print('Loss at epoch %d: %f' %(i+1, loss_fn(X, Y).numpy()))
            
fit(x_data, y_data)

Loss at epoch 1: 0.159370
Loss at epoch 100: 0.153950
Loss at epoch 200: 0.148822
Loss at epoch 300: 0.144014
Loss at epoch 400: 0.139495
Loss at epoch 500: 0.135241
Loss at epoch 600: 0.131231
Loss at epoch 700: 0.127443
Loss at epoch 800: 0.123861
Loss at epoch 900: 0.120468
Loss at epoch 1000: 0.117251
Loss at epoch 1100: 0.114196
Loss at epoch 1200: 0.111291
Loss at epoch 1300: 0.108526
Loss at epoch 1400: 0.105891
Loss at epoch 1500: 0.103378
Loss at epoch 1600: 0.100977
Loss at epoch 1700: 0.098683
Loss at epoch 1800: 0.096488
Loss at epoch 1900: 0.094386
Loss at epoch 2000: 0.092371


# Class Version

In [81]:
class softmax_classifer(tf.keras.Model):
    def __init__(self, nb_classes):
        super(softmax_classifer, self).__init__()
        self.W = tfe.Variable(tf.random_normal([4, nb_classes]), name='weight')
        self.b = tfe.Variable(tf.random_normal([nb_classes]), name='bias')
        
    def softmax_regression(self, X):
        return tf.nn.softmax(tf.matmul(X, self.W) + self.b)
    
    def loss_fn(self, X, Y):
        logits = self.softmax_regression(X)
        loss = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(logits), axis=1))
        
        return loss
    
    def grad_fn(self, X, Y):
        with tf.GradientTape() as tape:
            loss = self.loss_fn(x_data, y_data)
            grads = tape.gradient(loss, self.variables)
            
            return grads
    
def fit(self, X, Y, epochs=2000, verbose=500):
    optimizer =  tf.train.GradientDescentOptimizer(learning_rate=0.1)

    for i in range(epochs):
        grads = self.grad_fn(X, Y)
        optimizer.apply_gradients(zip(grads, self.variables))
        if (i==0) | ((i+1)%verbose==0):
            print('Loss at epoch %d: %f' %(i+1, self.loss_fn(X, Y).numpy()))

In [82]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
model = softmax_classifer(nb_classes)

In [83]:
model.fit(x_data, y_data)

Loss at epoch 1: 6.807235
Loss at epoch 500: 0.478263
Loss at epoch 1000: 0.254608
Loss at epoch 1500: 0.194818
Loss at epoch 2000: 0.159951
